# 数据处理

In [31]:
%cd /Users/cf/Documents/工程实践
! ls

region="106"

/Users/cf/Documents/工程实践
UrbanEV             UrbanEVDataset      UrbanEVSupplemental analyze


In [32]:
from pathlib import Path
import pandas as pd
from math import radians, sin, cos, sqrt, atan2, pi

def read_col(path: Path, region: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    if "time" not in df.columns or region not in df.columns:
        raise ValueError(f"Missing columns in {path}")
    return df[["time", region]].copy()

def haversine(lon1, lat1, lon2, lat2):
    R = 6371000.0
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlambda/2)**2
    return 2*R*atan2(sqrt(a), sqrt(1-a))

def process(data:Path,region:str,outdir:Path):
    base = data
    occ = read_col(base / "occupancy.csv", region)
    dur = read_col(base / "duration.csv", region)
    vol = read_col(base / "volume.csv", region)
    vol11 = read_col(base / "volume-11kW.csv", region)

    occ.rename(columns={region: "occupancy"}, inplace=True)
    dur.rename(columns={region: "duration"}, inplace=True)
    vol.rename(columns={region: "volume"}, inplace=True)
    vol11.rename(columns={region: "volume-11kW"}, inplace=True)

    out = occ.merge(dur, on="time").merge(vol, on="time").merge(vol11, on="time")
    outdir.mkdir(parents=True, exist_ok=True)
    out.to_csv(outdir/f"region-{region}.csv", index=False)
def process2(data:Path,region:str,outdir:Path):
    base = data
    occ = read_col(base / "occupancy.csv", region)
    dur = read_col(base / "duration.csv", region)
    vol = read_col(base / "volume.csv", region)
    vol11 = read_col(base / "volume-11kW.csv", region)
    s_price = read_col(base / "s_price.csv", region)
    e_price = read_col(base / "e_price.csv", region)

    s_price.rename(columns={region: "s_price"}, inplace=True)
    e_price.rename(columns={region: "e_price"}, inplace=True)
    occ.rename(columns={region: "occupancy"}, inplace=True)
    dur.rename(columns={region: "duration"}, inplace=True)
    vol.rename(columns={region: "volume"}, inplace=True)
    vol11.rename(columns={region: "volume-11kW"}, inplace=True)

    out = occ.merge(dur, on="time").merge(vol, on="time").merge(vol11, on="time").merge(s_price, on="time").merge(e_price, on="time")
    outdir.mkdir(parents=True, exist_ok=True)
    out.to_csv(outdir/f"region-{region}-all.csv", index=False)
    


In [33]:
def show(region:str):
    df = pd.read_csv(region)
    t = pd.to_datetime(df["time"]) 
    p = Path(region)
    rid = p.name.split('-')[-1].split('.')[0]
    base = Path("analyze/data")
    summary_rows = []
    inf = pd.read_csv(base / "inf.csv")
    m = inf[inf["TAZID"].astype(str) == str(rid)]
    row = m.iloc[0]
    print(f"region={rid} info: longitude={row['longitude']}, latitude={row['latitude']}, charge_count={row['charge_count']}, area={row['area']}, perimeter={row['perimeter']}")
    summary_rows.append({"section": "region_info", "name": "longitude", "value": row['longitude']})
    summary_rows.append({"section": "region_info", "name": "latitude", "value": row['latitude']})
    summary_rows.append({"section": "region_info", "name": "charge_count", "value": row['charge_count']})
    summary_rows.append({"section": "region_info", "name": "area", "value": row['area']})
    summary_rows.append({"section": "region_info", "name": "perimeter", "value": row['perimeter']})
    poi = pd.read_csv(base / "poi.csv")
    lon0 = float(row["longitude"])
    lat0 = float(row["latitude"])
    area_v = float(row["area"])
    radius = sqrt(area_v / pi)
    dists = poi.apply(lambda r: haversine(lon0, lat0, r["longitude"], r["latitude"]), axis=1)
    inside = poi[dists <= radius]
    counts = inside.groupby("primary_types").size().reset_index(name="count")
    print(counts.to_string(index=False))
    for _, r in counts.iterrows():
        summary_rows.append({"section": "poi_count", "name": r['primary_types'], "value": r['count']})
    print(f"列名: {list(df.columns)}")
    print(f"条数={df.shape[0]}")
    print(f"时间范围={t.min()} 到 {t.max()}")
    summary_rows.append({"section": "data_summary", "name": "columns", "value": '|'.join(list(df.columns))})
    summary_rows.append({"section": "data_summary", "name": "row_count", "value": df.shape[0]})
    summary_rows.append({"section": "data_summary", "name": "time_start", "value": str(t.min())})
    summary_rows.append({"section": "data_summary", "name": "time_end", "value": str(t.max())})
    pd.DataFrame(summary_rows).to_csv(p.parent / f"region-{rid}-summary.csv", index=False)


In [34]:
#提取102区域的time,occupancy, duration, volume,volume-kw 数据
#输出region-102.csv
process(Path("analyze/data"),region,Path(f"analyze/result/region-{region}"))
#提取102区域的time,occupancy, duration, volume,volume-kw,s_price,e_price 数据
#输出region-102-all.csv
process2(Path("analyze/data"),region,Path(f"analyze/result/region-{region}"))


In [35]:
#查看该区域大小,输出此区域的primary_types,存储在region-102-summary.csv
show(f"analyze/result/region-{region}/region-{region}.csv")


region=106 info: longitude=114.1218804, latitude=22.5442716, charge_count=114.0, area=578717.374, perimeter=3486.9251
             primary_types  count
  business and residential    293
food and beverage services    360
        lifestyle services    900
列名: ['time', 'occupancy', 'duration', 'volume', 'volume-11kW']
条数=4344
时间范围=2022-09-01 00:00:00 到 2023-02-28 23:00:00


# Todo:区域级画像分析
# 1.数据统计
- 小时级别全城充电总量、占用率,时长的统计规律
- 天级别全城充电总量、占用率,时长的统计规律
  - 节假日级别全城充电总量、占用率,时长的统计规律
  - 周一到周末级别全城充电总量、占用率,时长的统计规律

In [36]:
# 查看该区域每个小时的occupancy, duration, and volume数据
# 输出region-102-hourly.csv
# 绘制occupancy, duration, and volume的折线图
from analyze import analyze_region_hourly
analyze_region_hourly.analyze(Path(f"analyze/result/region-{region}/region-{region}.csv"),Path(f"analyze/result/region-{region}"),region)

总计T天=181
小时均值(部分预览):
   hour  volume_mean  occupancy_mean  duration_mean
0     0    86.997238       48.980663      12.428177
1     1    89.607735       49.839779      12.801105
2     2    90.854972       49.259669      12.979282
3     3    89.208103       50.041436      12.744015
4     4    87.593462       49.342541      12.513352
工作日中午(12-13) volume均值=89.91, 周末=83.77
工作日中午(12-13) occupancy均值=50.62, 周末=49.79
工作日中午(12-13) duration均值=12.84, 周末=11.97


In [37]:
# 查看该区域每周的occupancy, duration, and volume数据
# 输出region-102-weekly.csv
# 绘制occupancy, duration, and volume的折线图
from analyze import analyze_region_weekly
analyze_region_weekly.analyze(Path(f"analyze/result/region-{region}/region-{region}.csv"),Path(f"analyze/result/region-{region}"),region)

总计T天=181
周统计预览:
  name  vol_sum_mean  occ_mean_mean  dur_mean_mean
0  Mon   1994.394231      48.990385      11.871394
1  Tue   1946.134615      48.863782      11.584135
2  Wed   1922.970000      48.821667      11.446250
3  Thu   1999.980769      49.329327      11.904647
4  Fri   1975.817308      49.408654      11.760817
5  Sat   2128.987179      49.406250      12.672543
6  Sun   2060.737179      49.286058      12.266293
周一均值: volume日总=1994.39, occupancy日均=48.99, duration日均=11.87


In [39]:
#节假日表是手动生成的,放在analyze/data/holidays.csv里
# 查看该区域节假日与普通日期的occupancy, duration, and volume数据
# 输出region-102-holiday.csv
# 绘制occupancy, duration, and volume的箱体图
from analyze import analyze_region_holiday
analyze_region_holiday.analyze(Path(f"analyze/result/region-{region}/region-{region}.csv"),Path(f"analyze/result/region-{region}"),Path(f"analyze/data/holidays.csv"),region)



总计T天=181
工作日天数=161, 节假日天数=20
   is_holiday  vol_sum_mean  occ_mean_mean  dur_mean_mean
0       False   2004.336957      49.250259      11.930577
1        True   2006.666667      48.432292      11.944444


# 2.聚类(需要再思考一下聚类可行性)

In [40]:
#按天聚类
from analyze import analyze_region_cluster
analyze_region_cluster.analyze(Path(f"analyze/result/region-{region}/region-{region}-all.csv"),Path(f"analyze/result/region-{region}/cluster"), region)
#- cluster 0：夜间补能型
# - 夜间 23 点附近同时出现 volume 与 duration 峰，说明夜间补能、停留更久。
# - 与高价 e_price 呈明显负相关，低价时段更活跃。
# - cluster 1：白天通勤/运营型
# - 上午 9–10 点出现 volume/occupancy/duration 峰，白天时段需求更强。
# - 平均规模更大（量、高占用、长时长），代表日间活跃的主力需求。


聚类簇数k=2
示例簇均值预览：
   cluster  hour      volume  occupancy   duration  volume-11kW   s_price  \
0        0     0   99.705556  50.404762  14.243651   100.247222  0.633333   
1        0     1  103.461111  51.723810  14.780159   103.461111  0.633333   
2        0     2  105.216667  50.980952  15.030952   105.216667  0.633333   
3        0     3  103.922222  52.080952  14.846032   103.922222  0.633333   
4        0     4  102.350000  51.152381  14.621429   102.350000  0.633333   

   e_price  
0     1.17  
1     1.17  
2     1.17  
3     1.17  
4     1.17  


In [42]:
analyze_region_cluster.analyze_by_hour(Path(f"analyze/result/region-{region}/region-{region}-all.csv"),Path(f"analyze/result/region-{region}/cluster2"),region)


按小时聚类簇数k=2
   cluster      volume  occupancy   duration  volume-11kW   s_price  e_price
0        0   64.558826  46.492727   9.222689    64.616121  0.633333     1.17
1        1  102.986085  51.896688  14.712298   102.986085  0.633333     1.17


# Todo: 站点级分析
- 标注每个站点属于哪个区域,是什么类型的站点

In [44]:
from pathlib import Path
import numpy as np
def map_station_poi_taz(station_path: Path, poi_path: Path, out_path: Path):
    stations = pd.read_csv(station_path)
    poi = pd.read_csv(poi_path)
    lons_poi = poi['longitude'].values
    lats_poi = poi['latitude'].values
    types_poi = poi['primary_types'].values
    def nearest_type(lon, lat):
        d = np.array([haversine(lon, lat, lons_poi[i], lats_poi[i]) for i in range(len(lons_poi))])
        j = int(d.argmin())
        return types_poi[j]
    res = pd.DataFrame({
        'station_id': stations['station_id'],
        'longitude': stations['longitude'],
        'latitude': stations['latitude'],
    })
    res['primary_types'] = stations.apply(lambda r: nearest_type(float(r['longitude']), float(r['latitude'])), axis=1)
    res['TAZID'] = stations['TAZID']
    outdir = out_path if not out_path.suffix else out_path.parent
    if outdir.exists() and outdir.is_file():
        outdir = outdir.parent
    outdir.mkdir(parents=True, exist_ok=True)
    res.to_csv(outdir / 'station-poi-taz.csv', index=False)


In [45]:
map_station_poi_taz(Path(f"analyze/data/20220901-20230228_station-raw/station_information.csv"),Path(f"analyze/data/poi.csv"),Path(f"analyze/result/station"))


KeyboardInterrupt: 